# Neuron

Define a function `dot` which calculates the dot product of two vectors

In [2]:
dot x y = sum $ zipWith(*) x y

we use `zipWith` to multiply together the components in each vector and then we use sum to add all of the results.

In [3]:
dot [1,2] [3,4]

11

Now we can use that to define a single neuron. A neuro has 3 properties that are interesting:
* It has input weights which they are numbers that scales the inputs
* It has a bias which is a single number that it's added to the input singals
* It has an activation function which controls what the output of the neuron is like

A neuron itself in Machine Learning terms is a function. 

In [4]:
neuron weights bias activation inputs = let b = dot weights inputs + bias in 
                                        activation b

To actually use a neuron we need to pick an activation function. There are 3 commons ones:
* **reLU**: If the input < 0 then output 0, otherwise output the input. 
* **Sigmoid**: If the input is very low then 0 if the input is very high then 1 and between the two of them there is a smooth S shape curve from 0 to 1 when the input is close to 0.
* **Tanh**: It's similar to the sigmoid but it has a different shape near the top and the bottom of the S. 

In [5]:
reLU x = if x < 0 then 0 else x

With this activation function we now have everything we need to actually use the neuron

In [6]:
mineneuron = neuron [1] 0 reLU

That's everything we should do to define a neuron. We now have a neuron where the `weight` is 1, the `bias` is 0 and the `activation function` is reLU

In [7]:
mineneuron [0]
mineneuron [1]
mineneuron [-1]

0

1

0

With this weight and bias the neuron does not do very much so let's make a more interesting neuron.

In [8]:
neuron2 = neuron [2] 1 reLU

And now we test our neuron with some inputs

In [9]:
neuron2 [0]
neuron2 [1]
neuron2 [-1]

1

3

0

Our neuron with a simple input is doing `output = weight * input + bias`. This is the same formula as the formula for an straight line `y = m * x + b`. So, our neuron is modelling a straigh line by changing the `weight` and the `bias` we change the line. Also the activation function means that we do not output just the line but instead we clamp values where `y < 0`. If we just do a graph `y = 2x + 1` then it would be a straigh line the goes both up and to the right but also down onto the left and the `activitation function` means that we stop the neuron going down onto the left. Withou the activation function could only ever describe a straigh line so we use activation functions to let the neuron descrive more complex shapes like curves.

In [10]:
multiweightneuron = neuron [2,3,5,6] 0 reLU

In [19]:
multiweightneuron [1,2,3,4]
multiweightneuron [10,20,30,40]
multiweightneuron [-1,0,2,3]
multiweightneuron [1,0,0,0]

47

470

26

2

With the last of those because all the inputs are `0` apart of one of them it is like we are selecting one of the `weights` to output by only passing a 1 for that input. 

In [11]:
multiweightneuron [1,0,0,0]
multiweightneuron [0,1,0,0]
multiweightneuron [0,0,1,0]
multiweightneuron [0,0,0,1]

2

3

5

6

The `outputs` that we get are the `weight` values. this is because by setting the other inputs to 0 we make sure that all of the other weights get multiply into nothing so only the weight for the input we said one will be included in the `output`.

## Simple classifier

We can use one Neuron as a simple classifier that decides whether an input is in one class or in another. For the purpose of this example, we want the Neuron to output a `0` for inputs in `classHaveFun` and a `1` for inputs in `classGetBored`. The input will be 3 variables: `isThereVideogames`, `isThereAnime` and `pendingAdultingTasks`. The neuron will take this 3 variables as inputs it will compute the weighted sum and then convereted to a `0` or `1` using the `Sigmoid` activation function.

In [15]:
-- Build the Sigmoid function
sigmoid x = 1/(1 + exp(-x))

In [24]:
sigmoid 0

0.5

Now that we have the sigmoid function we can make a neuron with it:

In [26]:
funCalculator = neuron [2,1,-1] 0 sigmoid

In [27]:
funCalculator [1,0,3]

0.2689414213699951

The `output` value is close to `0` which it means that this `input` would be classed as `classGetBored`... So unfortunately not so much fun.

In [28]:
funCalculator [1,1,0]

0.9525741268224334

So, when we do not have `pendingAdultingTasks` we see that the `output` is close to `1` which it means that this `input` will be classified as `classHaveFun`

In [29]:
funCalculator [1,1,1]

0.8807970779778823

In that case when we have `isThereVideogames`, `isThereAnime` and only 1 `pendingAdultingTasks` it's a little bit less fun... But still fun!

In [30]:
funCalculator [1,1,3]

0.5

In this last example, the `output` is `0.5` which is exactly between the two classes, it's equally fun and not fun. 

Suppose that we decide, when we think about how the situation we are describing actually works in reality, that when a person has video games and anime but there are `3 `pending adulting tasks to take care of, in reality they would decide that situation
is definitely boring. So the answer our model neuron is giving in this situation is not quite what we want because it does not match reality: we want the input `[1, 1, 3]` to give an output which is less than `0.5`, and we want to change our model so that
this happens.

We can ajust the neuron by changing the `bias` just a little bit. If we set the bias to `-0.5` it woul dmake the neuron treat all situations as slightly less fun which will break the tie between the two classes for this `input`. 

In [32]:
funCalculatorMod = neuron [2,1,-1] (-0.5) sigmoid

In [33]:
funCalculatorMod [1,1,3]

0.3775406687981454

Now we can see that the `input` produces an output which is more clearly in the `classGetBored`.

Now, we will test the other examples and see the difference with our new modified neuron:

In [34]:
funCalculatorMod [1,0,3]
funCalculatorMod [1,1,0]
funCalculatorMod [1,1,1]

0.18242552380635635

0.9241418199787566

0.8175744761936437

We observe slightly lower numbers but each one is still close to the same classes before. This means that we were able to change the class for the one input we wanted it without affecting the others. So we dicreseat the `error` for our model. We went from `3/4` to `4/4` or from `75%` to `100%` accuracy . 

## Logic gates

We can also make a neuron behave like a simple logic gate. Suppose we want to make a neuron which behaves like an AND gate:

| A | B | Output |
|---|---|--------|
| 0 | 0 | 0      |
| 0 | 1 | 0      |
| 1 | 0 | 0      |
| 1 | 1 | 1      |

We can do this using one neuron, which takes two inputs. For the output, we want it to be less than 0.05 ('close enough' to 0) or greater than 0.95 ('close enough' to 1). So we can make our neuron like this: 

Firstly, we can pick an `activation function` because the outputs in this case are `0` and `1` it's a strong hint that we need to be using `sigmoid` because `sigmoid` output values that are most 0 or 1. `reLU` would not be a good option because while it produces 0 for negative values it will produce large outputs for positive values so, it won't be just `0` and `1`. `tanH` is also not ideal because `tanH` outputs are between `-1` and `1`. So, for this kind of problem we want a function that outputs between 2 classes `1` and `0`, `sigmoid` is usually the best choise. 

Now, for the bias the simple thing to do it's to start with `bias` at `0` if it's `0` means that it will not be affected at all. So, focus on the `weights` and then we will come back to the `bias` later if we need to. 

For the `weights` we start both at `1` because we need them something that it's not `0` - because if it was `0` the input just will be ignore because the value is multipled by the weight - and also we gave equal `weights` because ther eis no reason to think that `1` should be different to the other. In the example of the `andGate` the inputs should be interchangable so the weight we expect will be equal. 

In [45]:
andGate = neuron [1, 1] 0 sigmoid

andGate [0, 0] -- should be < 0.05
andGate [0, 1] -- should be < 0.05
andGate [1, 0] -- should be < 0.05
andGate [1, 1] -- should be > 0.95

0.5

0.7310585786300049

0.7310585786300049

0.8807970779778823

So in that example, what we see is that in the cases where the output should be `0` we are getting values that they are too high. And in the case that should be `1` the value is too low. So what we do next is just increase the `weight` to scale everything up and push the output further away from the middle. 

In [46]:
andGate = neuron [100, 100] 0 sigmoid

andGate [0, 0] -- should be < 0.05
andGate [0, 1] -- should be < 0.05
andGate [1, 0] -- should be < 0.05
andGate [1, 1] -- should be > 0.95

0.5

1.0

1.0

1.0

Now the last case it's correct but the first 3 should be `0`. now, we think about the linear function in the `neuron`. We want the linear part to be less than 0 so that `sigmoid` makes it be a `0`.  

`0*100 + 0*100 + 0 = 0
0*100 + 1*100 + 0 = 100
1*100 + 0*100 + 0 = 100
1*100 + 1*100 + 0 = 200`

We want the first 3 values to be below `0` but the last value still above 0. So, if we use a `bias` of `-150` that should get what we want when is add to this numbers.   


In [47]:
andGate = neuron [100, 100] (-150) sigmoid

andGate [0, 0] -- should be < 0.05
andGate [0, 1] -- should be < 0.05
andGate [1, 0] -- should be < 0.05
andGate [1, 1] -- should be > 0.95

7.175095973164411e-66

1.928749847963918e-22

1.928749847963918e-22

1.0

Now we have 3 very small numbers and one that close to `1` which is what we wanted. Puzzel solve! 

We could also make an OR gate:

| A | B | Output |
|---|---|--------|
| 0 | 0 |   0    |
| 0 | 1 |   1    |
| 1 | 0 |   1    |
| 1 | 1 |   1    |

Again we can do this with one neuron, which takes two inputs, and the output is < 0.05 or 0.95:

In [48]:
orGate = neuron [100, 100] (-150) sigmoid

orGate [0, 0] -- should be < 0.05
orGate [0, 1] -- should be > 0.95
orGate [1, 0] -- should be > 0.95
orGate [1, 1] -- should be > 0.95

7.175095973164411e-66

1.928749847963918e-22

1.928749847963918e-22

1.0

In [ ]:
We cannot use the same for `orGate` because the behaviour is obviously different. 

`0*100 + 0*100 + 0 = 0
0*100 + 1*100 + 0 = 100
1*100 + 0*100 + 0 = 100
1*100 + 1*100 + 0 = 200`

It's the same but this time we only want the first number negative because only the first one should be a `0` output. We can pick a different `bias` now. 

In [49]:
orGate = neuron [100, 100] (-50) sigmoid

orGate [0, 0] -- should be < 0.05
orGate [0, 1] -- should be > 0.95
orGate [1, 0] -- should be > 0.95
orGate [1, 1] -- should be > 0.95

1.928749847963918e-22

1.0

1.0

1.0

We can see how changing the `bias` by reducing it 3 times, we can obtain the expected result. We change an `andGate` to an `orGate` just ajusting the `bias`

Thirdly we can do a NOT gate:

| A | Output |
|---|--------|
| 0 |   1    |
| 1 |   0    |

This time the neuron will only have one input, and again we make the output be < 0.05 or > 0.95:

In [ ]:
notGate = neuron [-100] 50 sigmoid

notGate [0] -- should be > 0.95
notGate [1] -- should be < 0.05 